In [1]:
from matplotlib import pyplot as plt
import h5py
import numpy as np
import matplotlib.gridspec as gridspec
import random
from astroML.correlation import two_point

z="{:.2f}".format(0.20) # repeat for all redshifts
cdm_galaxy = h5py.File('Data/n/home07/sbose/COLOR_CDM_Galaxies_z'+z+'.hdf5', 'r')
wdm_galaxy = h5py.File('Data/n/home07/sbose/COLOR_WDM_Galaxies_z'+z+'.hdf5', 'r')

print(cdm_galaxy,wdm_galaxy)

<HDF5 file "COLOR_CDM_Galaxies_z0.20.hdf5" (mode r)> <HDF5 file "COLOR_WDM_Galaxies_z0.20.hdf5" (mode r)>


# Access the masses and positions of CDM and WDM galaxies

In [2]:
# Access mass/position for wdm/cdm
print(list(cdm_galaxy.keys()),list(wdm_galaxy.keys()))

cdm_galaxy_color = cdm_galaxy['Colour'][:]
cdm_galaxy_position = cdm_galaxy['GalaxyPos'][:]
cdm_galaxy_mass = cdm_galaxy['StellarMass'][:]

wdm_galaxy_color = wdm_galaxy['Colour'][:]
wdm_galaxy_position = wdm_galaxy['GalaxyPos'][:]
wdm_galaxy_mass = wdm_galaxy['StellarMass'][:]

cdm_galaxy_color, cdm_galaxy_position, cdm_galaxy_mass, 
wdm_galaxy_color, wdm_galaxy_position, wdm_galaxy_mass

['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass'] ['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass']


(array([0.26351547, 0.26975822, 0.55730152, ..., 0.60256481, 0.60255599,
        0.60209143]), array([[18.13657761, 80.13793945, 52.1907692 ],
        [20.0150547 , 78.72826385, 53.95800018],
        [20.30582809, 79.4005127 , 53.66751862],
        ...,
        [78.52874756, 83.88411713, 62.66805267],
        [97.82990265, 36.0692482 , 37.34383011],
        [ 7.79859877, 48.99835968, 51.00095749]]), array([2.18818500e+06, 3.09918325e+06, 8.09408625e+05, ...,
        1.59784888e+03, 1.56134949e+03, 1.61065710e+03]))

# Compute the galaxy positions at fixed abundance
### Send the galaxy positions to Odyssey at n =  $\sim$0.33, $\sim$0.14, and 0.03 Galaxy/Mpc$^3$

In [3]:
# sort in order of descending mass
cdm_ind = sorted(range(len(cdm_galaxy_mass)), key=lambda k: cdm_galaxy_mass[k], reverse = True)
wdm_ind = sorted(range(len(wdm_galaxy_mass)), key=lambda k: wdm_galaxy_mass[k], reverse = True)

In [4]:
# iterate through each galaxy number density
for n in [326952,136834,30000]:
    
    # label the galaxy positions correctly
    label="_galaxy_position_z"+z+"_n{:.2f}".format(n/100**3)
    
    # save the galaxy positions
    np.savetxt("Calculations/cdm"+label+".txt", cdm_galaxy_position[cdm_ind[:n]], fmt = "%f %f %f")
    np.savetxt("Calculations/wdm"+label+".txt", wdm_galaxy_position[wdm_ind[:n]], fmt = "%f %f %f")

### Split the data by color!

In [5]:
# define the red and blue galaxies for CDM
cdm_blue = np.array(cdm_galaxy_color)[np.where(cdm_galaxy_color < 0.5)]
cdm_red = np.array(cdm_galaxy_color)[np.where(cdm_galaxy_color > 0.5)]

# define the red and blue galaxies for WDM
wdm_blue = np.array(wdm_galaxy_color)[np.where(wdm_galaxy_color < 0.5)]
wdm_red = np.array(wdm_galaxy_color)[np.where(wdm_galaxy_color > 0.5)]

In [6]:
# sort the blue galaxies in order of descending mass
cdm_blue_ind = sorted(range(len(cdm_blue)), key=lambda k: cdm_galaxy_mass[k], reverse = True)
wdm_blue_ind = sorted(range(len(wdm_blue)), key=lambda k: wdm_galaxy_mass[k], reverse = True)

# sort the red galaxies in order of descending mass
cdm_red_ind = sorted(range(len(cdm_red)), key=lambda k: cdm_galaxy_mass[k], reverse = True)
wdm_red_ind = sorted(range(len(wdm_red)), key=lambda k: wdm_galaxy_mass[k], reverse = True)

In [7]:
# obtain the positions of blue and red galaxies in CDM
cdm_blue_galaxy_position = np.array(cdm_galaxy_position)[cdm_blue_ind[:n]]
cdm_red_galaxy_position = np.array(cdm_galaxy_position)[cdm_red_ind[:n]]

# obtain the positions of blue and red galaxies in WDM
wdm_blue_galaxy_position = np.array(wdm_galaxy_position)[wdm_blue_ind[:n]]
wdm_red_galaxy_position = np.array(wdm_galaxy_position)[wdm_red_ind[:n]]

In [8]:
# iterate through each galaxy number density
for n in [326952,136834,30000]:
    
    # label the galaxy positions correctly
    label="_galaxy_position_z"+z+"_n{:.2f}".format(n/100**3)

    # save the blue galaxy positions
    np.savetxt("Calculations/cdm_blue"+label+".txt", cdm_blue_galaxy_position, fmt = "%f %f %f")
    np.savetxt("Calculations/wdm_blue"+label+".txt", wdm_blue_galaxy_position, fmt = "%f %f %f")

    # save the red galaxy positions
    np.savetxt("Calculations/cdm_red"+label+".txt", cdm_red_galaxy_position, fmt = "%f %f %f")
    np.savetxt("Calculations/wdm_red"+label+".txt", wdm_red_galaxy_position, fmt = "%f %f %f")